# Глава 1. Полносвязные нейронные сети

## Часть 3. Примеры реальных задач

Три финансовые задачи:
1. Задача классификации (мошенничества с кредитными картами)
2. Задача регрессии (страховые потери)
3. Задача обучения без учителя с применением автоэнкодера (мошенничества с кредитными картами)

## 1. Задача классификации

* Набор данных, который мы собираемся использовать, можно скачать с Kaggle. 

* Он содержит данные об операциях с кредитными картами, которые произошли в течение двух дней, с 492 мошенничествами из 284 807 транзакций.

* Все переменные в наборе данных являются числовыми.

* Данные были преобразованы с использованием преобразования PCA по соображениям конфиденциальности. 

* Два признака, которые не были преобразованы, это Время и Сумма транзакции. Время содержит секунды, прошедшие между каждой транзакцией и первой транзакцией в наборе данных.

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42

In [ ]:
LABELS = ["Normal", "Fraud"]

In [ ]:
df = pd.read_csv("datasets/creditcard.csv")

In [ ]:
df.shape

In [ ]:
df.isnull().values.any()

In [ ]:
count_classes = pd.value_counts(df['Class'], sort = True)
count_classes.plot(kind = 'bar', rot=0)
plt.title("Transaction class distribution")
plt.xticks(range(2), LABELS)
plt.xlabel("Class")
plt.ylabel("Frequency");

In [ ]:
frauds = df[df.Class == 1]
normal = df[df.Class == 0]
frauds.shape

In [ ]:
frauds.Amount.describe()

In [ ]:
f = plt.figure(figsize=(20,10))
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
f.suptitle('Amount per transaction by class')

bins = 50

ax1.hist(frauds.Amount, bins = bins)
ax1.set_title('Fraud')

ax2.hist(normal.Amount, bins = bins)
ax2.set_title('Normal')

plt.xlabel('Amount ($)')
plt.ylabel('Number of Transactions')
plt.xlim((0, 20000))
plt.yscale('log')
plt.show();

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
f.suptitle('Time of transaction vs Amount by class')

ax1.scatter(frauds.Time, frauds.Amount)
ax1.set_title('Fraud')

ax2.scatter(normal.Time, normal.Amount)
ax2.set_title('Normal')

plt.xlabel('Time (in Seconds)')
plt.ylabel('Amount')
plt.show()

In [ ]:
train, test = train_test_split(df, test_size = 0.2, stratify=df.Class)

In [ ]:
x_train = train.iloc[:,1:30].values[:]
y_train = train['Class'].values[:]

x_test = test.iloc[:,1:30].values[:]
y_test = test['Class'].values[:]

print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

### 1.3 Функция активации ReLU

ReLU является линейным для всех положительных значений и нулем для всех отрицательных значений. Это означает, что:

* Просто с вычислительной точки зрения. Таким образом, модели может потребоваться меньше времени для обучения.

* Быстрее сходимость. Линейность означает, что наклон не является плато или промежутком насыщения, когда x становится большим. 

* У ReLu нет проблемы исчезающего градиента, от которой страдают другие функции активации, такие как сигмоид или гиперболический тангес.

* Поскольку ReLU равен нулю для всех отрицательных входов. Производная также равна нулю. Бороться с этой проблемой, мы можем использовать негерметичный ReLU. Leaky ReLU гарантирует, что наклон для отрицательных значений не равен нулю.

![title](img/ann20.png)

### 1.4 Прореживание нейронной сети Dropout

**Переобучение (overfitting)** — одна из проблем глубоких нейронных сетей (Deep Neural Networks, DNN), состоящая в следующем: модель хорошо объясняет только примеры из обучающей выборки, адаптируясь к обучающим примерам, вместо того чтобы учиться классифицировать примеры, не участвовавшие в обучении (теряя способность к обобщению). 

* Главная идея Dropout — вместо обучения одной DNN обучить ансамбль нескольких DNN, а затем усреднить полученные результаты.

* Сети для обучения получаются с помощью исключения из сети (dropping out) нейронов с вероятностью , таким образом, вероятность того, что нейрон останется в сети, составляет . “Исключение” нейрона означает, что при любых входных данных или параметрах он возвращает 0.

* Исключенные нейроны не вносят свой вклад в процесс обучения ни на одном из этапов алгоритма обратного распространения ошибки (backpropagation); поэтому исключение хотя бы одного из нейронов равносильно обучению новой нейронной сети.

* В двух словах, Dropout хорошо работает на практике, потому что предотвращает взаимоадаптацию нейронов на этапе обучения.

![title](img/ann19.jpeg)

### 1.5 Методы оптимизации нейронных сетей

https://habr.com/ru/post/318970/

* Стохастический градиентный спуск (SGD)

* Nesterov Accelerated Gradient

* Adagrad

* RMSPROP и Adadelta

* Adam 

* Я бы предложил держать качестве метода оптимизации по умолчанию **Adam**, потому что он выдаёт наилучшие результаты при минимальном подгоне параметров. 
 
* Когда сеть уже более-менее отлажена, можно попробовать метод Нестерова с разными параметрами. Иногда с помощью него можно добиться лучших результатов, но он сравнительно чувствителен к изменениям в сети. Плюс-минус пара слоёв и нужно искать новый оптимальный learning rate.

* Рассматривайте остальные алгоритмы и их параметры как ещё несколько ручек и тумблеров, которые можно подёргать в каких-то специальных случаях.

### 1.6 Бинарная кросс-энтропия или Log Loss

![title](img/ann21.png)

![title](img/ann22.png)

Описание доступных функций потерь в Keras: https://keras.io/losses/
Как добавить свою метрику качества?

### 1.6 Keras Callbacks

Вызываемые функции в Keras

* **ModelCheckpoint**

Эта вызываемая функция сохранит вашу модель в виде файла контрольной точки (в формате hdf5) на диск после каждой успешной эпохи. Вы можете фактически установить выходной файл с динамическим именем в зависимости от эпохи. Вы также можете записать либо значение потерь, либо значение точности как часть имени файла журнала.

* **CSVLogger**

CSVLogger записывает файл истории в CSV, содержащий информацию об эпохах, точности и потерях на диск, чтобы вы могли проверить его позже.

* **EarlyStopping**

Одним из способов преодтвращения переобучения в нейронных сетях является использование ранней остановки. Ранняя остановка предотвращает переобучение вашей модели, прерывая процесс обучения, если процесс обучения более не улучшает обоющающую способность нейронной сети.

* **RemoteMonitor**

Этот возвращающая функция отправляет сообщения о состоянии в виде JSON через HTTP POST. Это может быть легко интегрировано со службой обмена сообщениями или с очередью, например Kafka, Amazon SQS и др.

* **LearningRateScheduler**

Другой метод оптимизации с глубоким обучением состоит в том, чтобы регулировать скорость обучения с течением времени. Скорость обучения определяет размером шагов градиентного спуска.

Один из методов - начать с относительно большого значения и уменьшать с увеличением эпох обучения. Все, что вам нужно сделать, это написать простую функцию, которая возвращает желаемую скорость обучения на основе текущей эпохи и передает ее в качестве параметра по расписанию (параметр schedule в callback LearningRateScheduler).

* **Tensorboard**

Это, пожалуй, самый крутой из всех стандартных callbacks. Используя callback TensorBoard, журналы будут записываться в каталог, который вы затем сможете просмотреть с помощью инструмента визуализации TensorBoard: кривые обучения, метрики качества и др.


In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(29,)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='SGD',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=500,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test))

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right');

In [ ]:
predictions = model.predict_proba(x_test)
y_pred = model.predict(x_test)
y_pred = (y_pred.flatten() > 0.5)*1
result_df = pd.DataFrame({'proba': predictions.flatten(), 'true_class': y_test})
result_df[result_df["true_class"]==1]

## 1.4. Кривая ROC. Площадь под ROC кривой - AUC ROC

In [ ]:
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)

In [ ]:
fpr, tpr, thresholds = roc_curve(result_df.true_class, result_df.proba)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show();

## 1.5. Кривая точность-полнота (Precision-recall curve)

![title](img/ann15.png)

![title](img/ann17.png)

![title](img/ann18.png)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
precision, recall, th = precision_recall_curve(result_df.true_class, result_df.proba)
plt.plot(recall, precision, 'b', label='Precision-Recall curve')
plt.title('Recall vs Precision')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

## 2. Задача регрессии 

Предсказание страхового требования к Страховой компании AllState

Каждая строка в этом наборе данных представляет страховое требование. Вы должны предсказать значение для столбца «потери». Переменные, начинающиеся с 'cat', являются категориальными, тогда как переменные, начинающиеся с 'cont', являются непрерывными.

https://www.kaggle.com/c/allstate-claims-severity

In [ ]:
from sklearn.feature_extraction import DictVectorizer

def prepare_data(data, train=True, dv=None):

    cat_keys = [k for k in data.keys() if k.startswith("cat")]
    cat_x = data[cat_keys]
    cont_keys = [k for k in data.keys() if k.startswith("cont")]
    cont_x = data[cont_keys]
    if train:
        y = data["loss"]
    else:
        y = None
    cat_x_dict = [r[1].to_dict() for r in cat_x.iterrows()]
    del cat_x
    if dv is None:
        dv = DictVectorizer().fit(cat_x_dict)
    cat_cont_x = dv.transform(cat_x_dict).toarray()
    del cat_x_dict
    return np.column_stack([cat_cont_x, cont_x]), y, dv

In [ ]:
train_data = pd.read_csv("datasets/train.csv").set_index("id")
train_data.head(5)

In [ ]:
print("Number of missing values", train_data.isnull().sum().sum())

Целевая переменная перекошена (ассиметрия)

In [ ]:
plt.figure(figsize=(13,9))
sns.distplot(train_data["loss"])
sns.boxplot(train_data["loss"])

Выглядит так, что распределение мы смогли нормализовать. Таким образом, мы будем использовать логарифм целевой переменной, потому что это дает возможность побороть выбросы

In [ ]:
plt.figure(figsize=(13,9))
sns.distplot(np.log1p(train_data["loss"]))

In [ ]:
train_x, train_y, dict_vec = prepare_data(train_data) 

In [ ]:
test_data = pd.read_csv("datasets/test.csv").set_index("id")
test_x, _, _ = prepare_data(test_data, False, dict_vec)

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.objectives import MSE, MAE
from keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

seed=2019

In [ ]:
def baseline_model():
    model = Sequential()
    model.add(Dense(input_dim=train_x.shape[1], output_dim=256))
    model.add(Activation("tanh"))
    model.add(Dropout(0.50))
    model.add(Dense(output_dim=128))
    model.add(Activation("relu"))
    model.add(Dropout(0.50))
    model.add(Dense(output_dim=64))
    model.add(Activation("relu"))
    model.add(Dropout(0.50))
    model.add(Dense(output_dim=1))
    model.compile(optimizer="Nadam", loss="mean_absolute_error")
    return model

10% на валидацию для контроля early stopping

In [ ]:
model = baseline_model()

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

train_log = model.fit(train_x, train_y.values, batch_size=256, nb_epoch=200, validation_split=0.1, 
                      verbose=2, callbacks=[early_stopping])

In [ ]:
plt.plot(train_log.history["loss"], label="loss")
plt.plot(train_log.history["val_loss"], label="val_loss")
plt.legend()

In [ ]:
pred_y = model.predict(test_x)
result = pd.DataFrame(pred_y, index=test_data.index, columns=["loss"])
result.to_csv("submission.csv")

## Кросс-валидация

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
seed=17
np.random.seed(seed)

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=5, batch_size=500, verbose=0)))
pipeline = Pipeline(estimators)

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, train_x, train_y, cv=kfold, scoring="neg_mean_absolute_error")
print("Metrics: %.2f (%.2f) MSE" % (results.mean(), results.std()))

##  3. Задача классификации на основе нейросетевого автоэнкодера

* **Нейросетевой автоэнкодер** – алгоритм машинного обучения без учителя. 

* Основная идея автоэнкодера заключается в **уменьшении размерности** путем отображения исходного пространства признаков в латентное пространство меньшей размерности (этап кодирования), а затем в реконструкции входных данных на основе латентного представления (этап декодирования). 

* Сокращая размерность пространства, мы тем самым обучаем модель запоминать только наиболее важную информацию, из которой можно восстановить первоначальные данные. 

* Это свойство достигается путем формирования структуры кодировщика в виде «горлышка бутылки» (bottleneck), значительного уменьшая размерность последнего скрытого слоя. 

* Целью автоэнкодера является минимизация ошибки реконструкции между входными и выходными данными. Чтобы уменьшить ошибку реконструкции, в процессе обучения мы движемся в обратном направлении по нейронной сети и обновляем веса.

* Веса нейронов обновляются в зависимости от того, насколько они вносят вклад в ошибку реконструкции.

* Автоэнкодер можно рассматривать как алгоритм сжатия данных.

* Нейросетевой автоэнкодер захватывает нелинейные особенности данных, поэтому имеет преимущество перед PCA.

![title](img/ann12.png)

![title](img/ann13.png)

### 3.1. Ошибка реконструкции

Мы оптимизируем параметры нашей модели автоэнкодера таким образом, чтобы функция ошибки - ошибка реконструкции модели был сведен к минимуму. На практике часто используется традиционная среднеквадратичная ошибка:

![title](img/ann14.png)

### 3.2. Подготовка данных

Во-первых, давайте отбросим столбец Time (не собираюсь его использовать) и используем StandardScaler от Scikit на Amount. Масштабирующее устройство удаляет среднее значение и масштабирует значения до единичной дисперсии:

In [ ]:
df = pd.read_csv("datasets/creditcard.csv")

In [ ]:
from sklearn.preprocessing import StandardScaler

data = df.drop(['Time'], axis=1)
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

Обучение автоэнкодера несколько отличается от того, что мы наблюдали ранее. 

Допустим, у вас есть набор данных, содержащий множество не мошеннических транзакций. Вы хотите обнаружить любую аномалию в новых транзакциях. Мы создадим эту ситуацию, обучив нашу модель только обычным транзакциям. 

Резервирование правильного класса в тестовом наборе даст нам возможность оценить производительность нашей модели. Мы зарезервируем 20% наших данных для тестирования:

In [ ]:
X_train, X_test = train_test_split(data, test_size=0.2, stratify=data.Class, random_state=seed)
X_train = X_train[X_train.Class == 0]
X_train = X_train.drop(['Class'], axis=1)
y_test = X_test['Class']
X_test = X_test.drop(['Class'], axis=1)
X_train = X_train.values
X_test = X_test.values
X_train.shape

### 3.3 Построение модели автоэнкодера

Построение модели
Наш Автоэнкодер использует 4 полностью связанных слоя с 14, 7, 7 и 29 нейронами соответственно. Первые два слоя используются для нашего кодера, последние два - для декодера. Кроме того, регуляризация L1 будет использоваться во время обучения:

In [ ]:
input_dim = X_train.shape[1]
encoding_dim = 256 # 14

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

Давайте обучим нашу модель 100 эпохам с размером партии 32 сэмпла и сохраним наиболее эффективную модель в файл. ModelCheckpoint, предоставляемый Keras, действительно удобен для таких задач. Кроме того, результаты обучения будут экспортированы в формате, понятном TensorBoard.

In [ ]:
import keras.backend as K

def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
nb_epoch = 100
batch_size = 32

autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=[r2_score])

checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)

tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)

In [ ]:
history = autoencoder.fit(X_train, X_train, epochs=nb_epoch, batch_size=batch_size,
                          shuffle=True, validation_data=(X_test, X_test), verbose=1, 
                          callbacks=[checkpointer, tensorboard]).history

In [ ]:
autoencoder = load_model('model.h5')

### 3.4 Оценка качества модели

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right');

In [ ]:
predictions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - predictions, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': y_test})
error_df.describe()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
normal_error_df = error_df[(error_df['true_class']== 0) & (error_df['reconstruction_error'] < 10)]
_ = ax.hist(normal_error_df.reconstruction_error.values, bins=10)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
fraud_error_df = error_df[error_df['true_class'] == 1]
_ = ax.hist(fraud_error_df.reconstruction_error.values, bins=10)

In [ ]:
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)

Кривые ROC являются очень полезным инструментом для понимания производительности двоичных классификаторов. Однако наш случай немного необычен. У нас очень несбалансированный набор данных. Тем не менее, давайте посмотрим на нашу кривую ROC:

Кривая ROC отображает истинную положительную частоту в сравнении с ложной положительной скоростью в зависимости от различных пороговых значений. По сути, мы хотим, чтобы синяя линия была как можно ближе к верхнему левому углу. В то время как наши результаты выглядят довольно хорошо, мы должны помнить о природе нашего набора данных. РПЦ не выглядит для нас очень полезным. Onward ...

In [ ]:
fpr, tpr, thresholds = roc_curve(error_df.true_class, error_df.reconstruction_error)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show();

### 3.5 Кривая точности и полноты

In [ ]:
precision, recall, th = precision_recall_curve(error_df.true_class, error_df.reconstruction_error)
plt.plot(recall, precision, 'b', label='Precision-Recall curve')
plt.title('Recall vs Precision')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
plt.plot(th, precision[1:], 'b', label='Threshold-Precision curve')
plt.title('Precision for different threshold values')
plt.xlabel('Threshold')
plt.ylabel('Precision')
plt.show()

Ошибка реконструкции растет, а рекол падает

In [ ]:
plt.plot(th, recall[1:], 'b', label='Threshold-Recall curve')
plt.title('Recall for different threshold values')
plt.xlabel('Reconstruction error')
plt.ylabel('Recall')
plt.show()

### 4. Прогноз 

Чтобы предсказать, является ли новая / невидимая транзакция нормальной или мошеннической, мы рассчитаем ошибку реконструкции из данных транзакции. Если ошибка больше, чем предопределенный порог, мы помечаем ее как мошенническую(так как наша модель должна иметь низкую ошибку при обычных транзакциях). Давайте выберем это значение:

In [ ]:
threshold = 3.5

In [ ]:
groups = error_df.groupby('true_class')
fig, ax = plt.subplots()

for name, group in groups:
    ax.plot(group.index, group.reconstruction_error, marker='o', ms=3.5, linestyle='',
            label= "Fraud" if name == 1 else "Normal")
ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
plt.title("Reconstruction error for different classes")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show();

In [ ]:
y_pred = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
conf_matrix = confusion_matrix(error_df.true_class, y_pred)
plt.figure(figsize=(12, 12))
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

Наша модель ловит много мошеннических случаев. Однако есть и особенность. Модель имеет большое количество ложных срабатываний. 
Однако мы можем гибко варьировать порог и контролировать число обнаруживаемых в зависимости, например, от количества, которые мы можем обработать / проверить.

###  Задание 1.

Задача опредления вероятности дефолта (неуплаты долга) по кредитной карте

https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset

* Постройте классификатор на основе полносвязной нейронной сети для определения вероятности дефолта по кредитной карте (невозврат долга).

* Экземляров данных довольно мало, поэтому не нужно упорствовать с увеличением глубины и ширины сети

* Обязательное кодирование категориальных признаков

* Обязательная стандартизация или масштабирование признаков

* Если применяете One Hot Encoding (бинарное кодирование), то обратите внимание на регуляризацию, чтобы предотвратить переобучение

* Постройте классификатор на основе автоэнкодера для решения этой же задачи аналогично примеру с мошенничествам по картам

Удачи!